In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import json

# Caricamento dei dati dal file JSON
with open('../RemovedStopWordData/ate_absita_training_extracted.json', 'r') as file:
    data_train = json.load(file)

with open('../RemovedStopWordData/ate_absita_gold_extracted.json', 'r') as file:
    data_test = json.load(file)


X_train = data_train['aspects']
y_train = data_train['polarities']

X_test = data_test['aspects']
y_test = data_test['polarities']


# Definizione del modello
class ABSAModel(nn.Module):
    def __init__(self, num_labels):
        super(ABSAModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        aspect_representation = outputs.last_hidden_state[:, 0, :]
        aspect_representation = self.dropout(aspect_representation)
        logits = self.fc(aspect_representation)
        return logits

# Funzione per creare il dataset
class ABSADataset(Dataset):
    def __init__(self, sentences, polarities):
        self.sentences = sentences
        self.polarities = polarities
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        polarity = self.polarities[idx]
        inputs = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()
        polarity_tensor = torch.tensor(polarity)
        return input_ids, attention_mask, polarity_tensor

# Preparazione dei dati
train_dataset = ABSADataset(X_train, y_train)
test_dataset = ABSADataset(X_test, y_test)

# Definizione dei parametri
num_labels = 2  # Positivo e negativo
batch_size = 16
epochs = 5
learning_rate = 2e-5

# Creazione dei data loader
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Creazione del modello
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ABSAModel(num_labels).to(device)

# Definizione della funzione di loss e dell'ottimizzatore
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# Addestramento del modello
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for input_ids, attention_mask, polarities in train_loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        polarities = polarities.to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, polarities.argmax(dim=1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}')

# Valutazione del modello
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for input_ids, attention_mask, polarities in test_loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        polarities = polarities.to(device)

        logits = model(input_ids, attention_mask)
        predictions = logits.argmax(dim=1)
        total_correct += (predictions == polarities.argmax(dim=1)).sum().item()
        total_samples += polarities.size(0)

accuracy = total_correct / total_samples
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: stack expects each tensor to be equal size, but got [3, 2] at entry 0 and [4, 2] at entry 1